# SmartHVAC

In [1]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [ ]:


class RealisticHVACSimulator:
    def __init__(
        self,
        room_length=5.0,  # meters
        room_width=4.0,   # meters
        room_height=2.5,  # meters
        initial_temp=22.0,
        initial_humidity=50.0,
        outdoor_temp=30.0,
        outdoor_humidity=60.0,
        thermal_load_profile=None
    ):
        self.room_length = room_length
        self.room_width = room_width
        self.room_height = room_height
        self.volume = room_length * room_width * room_height  # m^3

        self.temperature = initial_temp
        self.humidity = initial_humidity

        self.outdoor_temp = outdoor_temp
        self.outdoor_humidity = outdoor_humidity

        # Actuators
        self.blower_speed = 0.0      # 0 (off) to 1 (max)
        self.heating_coil = 0.0      # 0 (off) to 1 (max)
        self.chiller = 0.0           # 0 (off) to 1 (max)
        self.humidifier = 0.0        # 0 (off) to 1 (max)
        self.dehumidifier = 0.0      # 0 (off) to 1 (max)

        # Thermal load profile: function of time, returns load in kW
        self.thermal_load_profile = thermal_load_profile if thermal_load_profile else (lambda t: 0.0)
        self.time = 0.0

    def step(self, blower_speed, heating_coil, chiller, humidifier, dehumidifier, dt=1.0):
        # Clamp actuator values
        self.blower_speed = np.clip(blower_speed, 0, 1)
        self.heating_coil = np.clip(heating_coil, 0, 1)
        self.chiller = np.clip(chiller, 0, 1)
        self.humidifier = np.clip(humidifier, 0, 1)
        self.dehumidifier = np.clip(dehumidifier, 0, 1)

        # Room air mass (approximate, dry air density ~1.2 kg/m^3)
        air_mass = self.volume * 1.2  # kg

        # Heating/cooling power (kW), simple linear scaling
        heating_power = 5.0 * self.heating_coil  # max 5 kW
        cooling_power = 5.0 * self.chiller      # max 5 kW

        # Blower effect: increases air exchange with outside
        air_exchange = 0.1 + 0.4 * self.blower_speed  # 0.1 to 0.5 air changes per hour
        air_exchange_per_dt = air_exchange * dt / 3600.0  # fraction of air exchanged per dt (dt in seconds)

        # Thermal load (kW)
        thermal_load = self.thermal_load_profile(self.time)

        # Temperature dynamics (Q = mcΔT, 1 kW = 860 kcal/h)
        net_power = heating_power - cooling_power + thermal_load
        delta_temp = (net_power * 3600 * dt) / (air_mass * 1005)  # 1005 J/kg.K specific heat of air

        # Air exchange with outside
        temp_leak = air_exchange_per_dt * (self.outdoor_temp - self.temperature)

        self.temperature += delta_temp + temp_leak

        # Humidity dynamics (very simplified)
        humid_change = 2.0 * self.humidifier - 2.0 * self.dehumidifier  # % per hour at max
        humid_leak = air_exchange_per_dt * (self.outdoor_humidity - self.humidity)
        self.humidity += (humid_change * dt / 3600.0) + humid_leak

        # Clamp values
        self.temperature = np.clip(self.temperature, 10, 40)
        self.humidity = np.clip(self.humidity, 10, 90)

        self.time += dt

        return self.temperature, self.humidity

    def get_feedback(self):
        return {
            'temperature': self.temperature,
            'humidity': self.humidity,
            'blower_speed': self.blower_speed,
            'heating_coil': self.heating_coil,
            'chiller': self.chiller,
            'humidifier': self.humidifier,
            'dehumidifier': self.dehumidifier,
            'time': self.time
        }